<a href="https://colab.research.google.com/github/jskim0406/Demo_dev_with_langchain/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

requirements

In [ ]:
!pip install langchain
!pip install transformers
!pip install einops  # mpt 모델 다운로드에 필요
!pip install chromadb  # vector 검색을 위한 DB

src_data

In [ ]:
from langchain.docstore.document import Document
text = "example sentences"
docs = [Document(page_content=text)]

summarize

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.chains.summarize import load_summarize_chain

model_id = "mosaicml/mpt-7b-instruct"
hf_tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
pipe = pipeline("text-generation", model=hf_model, tokenizer=hf_tokenizer, max_new_tokens=10)
llm = HuggingFacePipeline(pipeline=pipe)

chain = load_summarize_chain(llm=llm, chain_type="map_reduce")
chain.run(docs)

QA

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.indexes import VectorstoreIndexCreator

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
emb = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

index = VectorstoreIndexCreator(embedding=emb).from_documents(docs)
query = "example qeustion"
index.query(query, llm=llm)